In [1]:
from qleo import *
import numpy as np

# Simple QPE Example

The quantum phase estimation algorithm is an important subroutine of many more complex quantum algorithm.

The QPE algorithm finds the eigenvalue of a unitary operator U such that $U |v⟩ = e2^{πiθ} |v⟩$ where $0 ≤ θ ≤ 1$ and
$|v⟩$ is an eigenvector of $U$ .

It uses two distinct registers of qubits, respectively of n and m qubits each.

![QPE Circuit](https://upload.wikimedia.org/wikipedia/commons/b/b4/PhaseCircuit.svg "Title")

In [2]:
phi = 1/3
n = 14
m = 1

## Initialize qubits

$$\ket{0}_1\ket{\psi}_2 \rightarrow \frac{1}{\sqrt{2^n}} (\ket{0}_1 + \ket{1}_1)^{\otimes n} \ket{\psi}_2$$

$$\ket{\phi} = \ket{1}$$

In [3]:
mqubits = range(n,n+m)
nqubits = range(n)

In [4]:
c = Circuit()

c.push(GateH(), nqubits)
c.push(GateX(), mqubits)

15-qubit circuit with 15 instructions:
├── H @ q[0]
├── H @ q[1]
├── H @ q[2]
├── H @ q[3]
├── H @ q[4]
├── H @ q[5]
├── H @ q[6]
├── H @ q[7]
├── H @ q[8]
├── H @ q[9]
├── H @ q[10]
├── H @ q[11]
├── H @ q[12]
├── H @ q[13]
└── X @ q[14]

## Apply controlled operation

$$\rightarrow \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n - 1} \mathrm{e}^{2\pi\imath \phi k} \ket{k}_1 \otimes \ket{\psi}_2$$

In [5]:
for (i,j) in enumerate(reversed(nqubits)):
    angle = 2*np.pi*(np.mod(phi*(2**i),1)+1)
    c.push(GateCP(angle), j, mqubits)

c

15-qubit circuit with 29 instructions:
├── H @ q[0]
├── H @ q[1]
├── H @ q[2]
├── H @ q[3]
├── H @ q[4]
├── H @ q[5]
├── H @ q[6]
├── H @ q[7]
├── H @ q[8]
├── H @ q[9]
├── H @ q[10]
├── H @ q[11]
├── H @ q[12]
├── H @ q[13]
├── X @ q[14]
├── CP(8.377580409572781) @ q[13], q[14]
├── CP(10.471975511965976) @ q[12], q[14]
├── CP(8.377580409572781) @ q[11], q[14]
├── CP(10.471975511965976) @ q[10], q[14]
⋮   ⋮
└── CP(10.471975511965026) @ q[0], q[14]

## Inverse QFT

$$\rightarrow \frac{1}{2^n} \sum_{x=0}^{2^n-1}\sum_{k=0}^{2^n-1} \mathrm{e}^{-\frac{2\pi i k}{2^n} (x - 2^n \phi)} \ket{x} \otimes \ket{\psi}$$

In [6]:
c.push(QFT(n).inverse(), *nqubits)

c

15-qubit circuit with 30 instructions:
├── H @ q[0]
├── H @ q[1]
├── H @ q[2]
├── H @ q[3]
├── H @ q[4]
├── H @ q[5]
├── H @ q[6]
├── H @ q[7]
├── H @ q[8]
├── H @ q[9]
├── H @ q[10]
├── H @ q[11]
├── H @ q[12]
├── H @ q[13]
├── X @ q[14]
├── CP(8.377580409572781) @ q[13], q[14]
├── CP(10.471975511965976) @ q[12], q[14]
├── CP(8.377580409572781) @ q[11], q[14]
├── CP(10.471975511965976) @ q[10], q[14]
⋮   ⋮
└── QFT† @ q[0,1,2,3,4,5,6,7,8,9,10,11,12,13]

## Measure first register

$$\rightarrow \ket{2^n\phi} \otimes \ket{\psi}$$

In [7]:
c.push(Measure(), nqubits, range(n))
c

15-qubit circuit with 44 instructions:
├── H @ q[0]
├── H @ q[1]
├── H @ q[2]
├── H @ q[3]
├── H @ q[4]
├── H @ q[5]
├── H @ q[6]
├── H @ q[7]
├── H @ q[8]
├── H @ q[9]
├── H @ q[10]
├── H @ q[11]
├── H @ q[12]
├── H @ q[13]
├── X @ q[14]
├── CP(8.377580409572781) @ q[13], q[14]
├── CP(10.471975511965976) @ q[12], q[14]
├── CP(8.377580409572781) @ q[11], q[14]
├── CP(10.471975511965976) @ q[10], q[14]
⋮   ⋮
└── M @ q[13], c[13]

## Execute the circuit

In [8]:
res = Qleo().execute(c)
res

QCSResults:
├── simulator: QLEO 0.1.22
├── timings:
│    ├── sample time: 0.00046794s
│    └── apply time: 0.005080854s
├── fidelity estimate: 1
├── average multi-qubit gate error estimate: 0
├── most sampled:
│    ├── bs"10101010101010" => 684
│    ├── bs"11010101010101" => 165
│    ├── bs"01010101010101" => 49
│    ├── bs"11101010101010" => 31
│    └── bs"00110101010101" => 12
├── 1 executions
├── 0 amplitudes
└── 1000 samples

# Estimate the Phase

In [9]:
hist = res.histogram()
maxvector = max(hist, key=hist.get)

print("Most frequent bit string is : ", maxvector.to01())
print("With ", hist[maxvector], " occurances")

# convert bitvector of the counting register to an integer value
int_value = int(maxvector.to01()[::-1],base=2)

est_phase = int_value/2**n # convert to phase

print("\nThe estimated phase is: ", est_phase)
print("The actual phase is: ", phi)

Most frequent bit string is :  10101010101010
With  684  occurances

The estimated phase is:  0.33331298828125
The actual phase is:  0.3333333333333333
